## Looping through the Clifford group

For erroneous devices and low logical qubit numbers it can be beneficial to implement _all_ (or a large number of) Clifford circuits directly instead of composing them from a generating set (like H+S+CX) in order to optimize the circuit depth. 

To facilitate this use case, our `Clifford` class implements conversions between symplectic Clifford matrices and special ids assigned uniquely to all Clifford elements (up to Pauli operators) for a given number of qubits. This assignment follows the work of Koenig _et. al._ (_J. Math. Phys. 55, 122202_, [arXiv:1406.2170](https://arxiv.org/abs/1406.2170)).



For a given qubit number $n$, the integers $0,\dots,|\mathcal{\tilde{C}}_n|-1$ enumerate all elements of the $n$-qubit Clifford group up to Pauli operators $\mathcal{\tilde{C}}_n$. The size of this group can be accessed via `Clifford.group_size_mod_pauli(n)`. 

Given a Clifford id, we can obtain a symplectic matrix by using the Clifford constructor:

In [1]:
import htlogicalgates as htlg

htlg.Clifford(123, num_qubits=3).symplectic_matrix

array([[1, 1, 1, 0, 1, 1],
       [0, 0, 1, 1, 1, 1],
       [0, 1, 0, 1, 1, 1],
       [1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 0, 1],
       [0, 1, 1, 1, 1, 0]])

Conversely, we can turn a `Circuit` object into its corresponding `Clifford` object and query its unique id. Be reminded that all circuits in this package are Clifford circuits.

In [2]:
circuit = htlg.Circuit(2)
circuit.h(0)

clifford = circuit.to_clifford()
print(f"Clifford id: {clifford.id}")

Clifford id: 1


For completeness we want to note that the Pauli part can also be identified by an integer.

In [3]:
clifford = htlg.Clifford(id=23, pauli_id=2, num_qubits=4)
print(f"Pauli id: {clifford.pauli_id}")

Pauli id: 2


### Tailor multiple gates using the integer representation

In [tutorial.ipynb](./tutorial.ipynb) we learned how to use `tailor_logical_gate` to tailor a logical circuit for a stabilizer code. 


Instead of calling this function multiple times for different logical gates, we can use `tailor_multiple_logical_gates`. This function takes a list of Clifford ids as described above. 
Plus, we avoid the overhead of creating a distinct _Gurobi_ model for each gate. 

In [4]:
connectivity = htlg.Connectivity("circular", num_qubits=4)
stab_code = htlg.StabilizerCode("4_2_2")

results = htlg.tailor_multiple_logical_gates(
    stab_code, 
    connectivity, 
    range(2), # e.g., range(Clifford.group_size_mod_pauli(2))
    num_cz_layers=2
)

The function returns a dictionary containing all tailored gates and meta information.

In [5]:
print(list(results["Meta"].keys()))
print(list(results["Gates"].keys()))

['Connectivity', 'Code', 'Number CZ layers', 'Time limit', 'Started', 'Ended']
[0, 1]


For example, we can query the used connectivity and stabilizer code.

In [6]:
connectivity = results["Meta"]["Connectivity"]
stab_code = results["Meta"]["Code"]

print("Connectivity matrix:\n", connectivity.matrix)
print()
print("Code parameters:\n[[n,k,d]] =",stab_code.nkd)

Connectivity matrix:
 [[0 1 0 1]
 [1 0 1 0]
 [0 1 0 1]
 [1 0 1 0]]

Code parameters:
[[n,k,d]] = (4, 2, 2)


On the other hand, we can query the circuits from the results dictionary by identifying different gates by the integer that represents the logical Clifford action.

In [10]:
for key, val in results["Gates"][1].items():
    print(key, ":", repr(val.__str__()))

Circuit : 'X 0\nX 1\nZ 2\nC_XYZ 3\nCZ 1 2\nCZ 2 3\nC_XYZ 2\nCZ 1 2\nCZ 2 3\nS 2\nC_ZYX 3\n'
Status : 'Optimal'
Runtime : '13.533999919891357'


The runtime is given in seconds and the status message is the same as for the function `tailor_logical_gate`.

We can save the dictionary to a .json-file automatically by specifying the `output_file` argument of the function `tailor_multiple_logical_gates`. Alternatively, we can use the functions `save_results_dictionary` to manually save the results to a .json-file.

In [11]:
filepath = "tutorial_results.json"
htlg.save_results_dictionary(results, filepath)

We can load the results dictionary by using the function `load_results_dictionary`.

In [12]:
results_loaded = htlg.load_results_dictionary(filepath)